In [5]:
import torch

In [6]:
# class SkipGramFoo(torch.nn.Module):
#   def __init__(self, voc: int, emb: int, _):
#     super().__init__()
#     self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
#     self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
#     self.sig = torch.nn.Sigmoid()

#   def forward(self, inpt, trgs, rand):
#     emb = self.emb(inpt)
#     ctx = self.ffw.weight[trgs]
#     rnd = self.ffw.weight[rand]
#     out = torch.bmm(ctx, emb.unsqueeze(-1)).squeeze()
#     rnd = torch.bmm(rnd, emb.unsqueeze(-1)).squeeze()
#     out = self.sig(out)
#     rnd = self.sig(rnd)
#     pst = -out.log().mean()
#     ngt = -(1 - rnd + 10**(-3)).log().mean()
#     return pst + ngt

In [7]:
# args = (len(words_to_ids), 64, 2)
# mFoo = SkipGramFoo(*args)
# print('mFoo', sum(p.numel() for p in mFoo.parameters()))
# opFoo = torch.optim.Adam(mFoo.parameters(), lr=0.003)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# %pip install torchinfo
# from torchinfo import summary

# mFoo

In [9]:
# windows = list(more_itertools.windowed(tokens, 3))
# inputs = [w[1] for w in windows]
# targets = [[w[0], w[2]] for w in windows]
# input_tensor = torch.LongTensor(inputs)
# target_tensor = torch.LongTensor(targets)
# dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)

In [10]:
# wandb.init(project='mlx6-word2vec', name='mFoo')
# mFoo.to(device)
# for epoch in range(2):
#   prgs = tqdm.tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)
#   for inpt, trgs in prgs:
#     inpt, trgs = inpt.to(device), trgs.to(device)
#     rand = torch.randint(0, len(words_to_ids), (inpt.size(0), 2)).to(device)
#     opFoo.zero_grad()
#     loss = mFoo(inpt, trgs, rand)
#     loss.backward()
#     opFoo.step()
#     wandb.log({'loss': loss.item()})

-----

In [11]:
#load the embedding matrix
import joblib
embedding_dict = joblib.load('embeds.pkl')

word_to_id = joblib.load('word_to_ids.pkl')
id_to_word = joblib.load('idx_to_word.pkl')

In [12]:
# embed_dim = 300
# embeddings_tensor = torch.zeros(len(word_to_id), embed_dim, dtype=torch.float32)  # or load your actual weights
embeddings_tensor = torch.tensor([i for i in embedding_dict.values()], dtype=torch.float32)

/var/folders/7k/nmbh9fxj4dsbx6d3jx2k1lvm0000gn/T/ipykernel_52265/2386894407.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  embeddings_tensor = torch.tensor([i for i in embedding_dict.values()], dtype=torch.float32)


In [13]:
embeddings_tensor.shape

torch.Size([1244025, 300])

In [14]:
import torch
import torch.nn as nn

In [15]:
class DocTower(nn.Module):
    def __init__(self, embed_dim, sequence_length, lstm_hidden_dim, output_dim):
        """
        :param vocab_size: Size of the vocabulary.
        :param embed_dim: Dimensionality of token embeddings.
        :param lstm_hidden_dim: Number of hidden units in the LSTM.
        :param output_dim: Final embedding dimension (common space).
        """
        super(DocTower, self).__init__()
        # Maps token indices to embeddings (each of size embed_dim), embed with a given embedding dictionary
        self.embedding = nn.Embedding.from_pretrained(embeddings_tensor, padding_idx=0, freeze=False)
        # LSTM that processes sequences of embeddings.
        # With batch_first=True, input shape: (batch_size, sequence_length, embed_dim)
        self.lstm = nn.LSTM(embed_dim, lstm_hidden_dim, batch_first=True)
        # Final projection to the common embedding space
        self.fc = nn.Linear(lstm_hidden_dim, output_dim)
    
    def forward(self, x):
        """
        :param x: Tensor of shape (batch_size, sequence_length) containing token indices.
        :return: Tensor of shape (batch_size, output_dim) representing the sentence embedding.
        """
        # Convert token indices to embeddings.
        # Output shape: (batch_size, sequence_length, embed_dim)
        x = self.embedding(x)
        print(x.shape)
        # The LSTM processes the sequence in order (along the sequence_length dimension)
        # 'output' has shape (batch_size, sequence_length, lstm_hidden_dim)
        # 'hn' (hidden state) has shape (num_layers, batch_size, lstm_hidden_dim)
        output, (hn, cn) = self.lstm(x)
        print(output.shape)
        # Use the final hidden state from the last LSTM layer.
        # This represents the entire sentence after processing all tokens.
        last_hidden = hn[-1]  # Shape: (batch_size, lstm_hidden_dim)
        
        # Project the LSTM output into the common embedding space.
        return self.fc(last_hidden)


In [16]:
class TwoTowerModel(nn.Module):
    def __init__(self, query_sequence_length, doc_sequence_length, output_dim, embed_dim, lstm_hidden_dim):
        super(TwoTowerModel, self).__init__()
        self.query_tower = DocTower(embed_dim, query_sequence_length, lstm_hidden_dim, output_dim)
        self.doc_tower = DocTower(embed_dim, doc_sequence_length, lstm_hidden_dim, output_dim)
        
    def forward(self, query, doc_positive, doc_negative):
        """
        :param query: Tensor of shape (batch_size, query_input_dim)
        :param positive_context: Tensor of shape (batch_size, seq_length) containing token indices
        :param candidate: Tensor of shape (batch_size, query_input_dim) for negative samples
        """
        query_embed = self.query_tower(query)
        positive_embed = self.doc_tower(doc_positive)
        negative_embed = self.doc_tower(doc_negative)
        return query_embed, positive_embed, negative_embed


In [17]:
#prepare the data for the query tower work on the query triplets
import json
with open('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/data/train_triples_sample.json') as f:
    data = json.load(f)

with open('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/eda/train_triples_v1.1.json') as f:
    train = json.load(f)

with open('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/eda/valid_triples_v1.1.json') as f:
    val = json.load(f)

In [18]:
average_sentence_length = sum(len(item['query'].split()) for item in data) / len(data)
print(average_sentence_length)#this before stemming and removing stopwords
average_context_length = sum(len(item['positive'].split()) for item in data) / len(data)
print(average_context_length)#this before stemming and removing stopwords

12.0
42.8


In [19]:
from tqdm import tqdm
from utils import tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliagoryachev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
#data preparartion

In [21]:
SENT_LEN = 12
CONTEXT_LEN = 50
from nltk.stem import PorterStemmer
import constants

punctuation_map = constants.punctuation_map

def pad_truncate(data, max_len):
    res = []
    for d in tqdm(data):
        clean_sent = tokenize(d, punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True)
        tokens = clean_sent[:max_len]
        tokens += ['<unk>'] * (max_len - len(tokens))
        res.append(tokens)
    return res


In [22]:
# padded_query = pad_truncate(data, 'query', SENT_LEN)
# padded_positive = pad_truncate(data, 'positive', CONTEXT_LEN)
# padded_negative = pad_truncate(data, 'negative', CONTEXT_LEN)

In [23]:
#convert to indices using the word_to_id dictionary
import joblib
word_to_id = joblib.load('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/eda/word_to_ids.pkl')

In [24]:
#convert to indices
def convert_to_indices(data, word_to_id):
    res = []
    for d in tqdm(data):
        res.append([word_to_id.get(w, word_to_id['<unk>']) for w in d])
    return res

In [89]:
# padded_query = convert_to_indices(padded_query, word_to_id)
# padded_positive = convert_to_indices(padded_positive, word_to_id)
# padded_negative = convert_to_indices(padded_negative, word_to_id)

In [90]:
# padded_query

In [91]:
# padded_positive

In [155]:
#Wrap in dataset and dataloader
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class QueryDataset(Dataset):
    def __init__(self, data, SENT_LEN, CONTEXT_LEN):
        self.queries = [d['query'] for d in data]
        self.positives = [d['positive'] for d in data]
        self.negatives = [d['negative'] for d in data]
        #pad truncates the data
        self.queries = pad_truncate(self.queries, SENT_LEN)
        self.positives = pad_truncate(self.positives, CONTEXT_LEN)
        self.negatives = pad_truncate(self.negatives, CONTEXT_LEN)
        #convert to indices
        self.queries = convert_to_indices(self.queries, word_to_id)
        self.positives = convert_to_indices(self.positives, word_to_id)
        self.negatives = convert_to_indices(self.negatives, word_to_id)
    
    def __len__(self):
        return len(self.queries)
    
    def __getitem__(self, idx):
        return torch.tensor(self.queries[idx], dtype=torch.long), \
                torch.tensor(self.positives[idx], dtype=torch.long), \
                torch.tensor(self.negatives[idx], dtype=torch.long)
    
    def collate_fn(batch):
        queries, positives, negatives = zip(*batch)
        queries = torch.stack(queries)
        positives = torch.stack(positives)
        negatives = torch.stack(negatives)
        return queries, positives, negatives
    

# Wrap in dataset and dataloader
train_dataset = QueryDataset(train[:320], SENT_LEN, CONTEXT_LEN)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=QueryDataset.collate_fn)

val_dataset = QueryDataset(val[:320], SENT_LEN, CONTEXT_LEN)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=QueryDataset.collate_fn)

100%|██████████| 320/320 [00:00<00:00, 22272.73it/s]


100%|██████████| 320/320 [00:00<00:00, 78780.14it/s]


In [156]:
len(train_dataset)

320

In [158]:
for i in train_loader:
    keep_one = i
    print(i)
    print(i[0].shape)
    print(i[1].shape)
    break

(tensor([[ 406126,  393570,  681503,  456929,  286699,  354713,       0,       0,
               0,       0,       0,       0],
        [  27549,  925304,  301028,  483917,  896005,       0,       0,       0,
               0,       0,       0,       0],
        [ 426508,  426150,  312891,       0,       0,       0,       0,       0,
               0,       0,       0,       0],
        [  27549,  925304,  301028,  483917,  896005,       0,       0,       0,
               0,       0,       0,       0],
        [ 426508,  426150,  312891,       0,       0,       0,       0,       0,
               0,       0,       0,       0],
        [1139912,  537817,       0,       0,       0,       0,       0,       0,
               0,       0,       0,       0],
        [ 686966,  798043,       0,       0,       0,       0,       0,       0,
               0,       0,       0,       0],
        [ 876576, 1218058,  948181,  655412,       0,       0,       0,       0,
               0,       0,   

In [173]:
idx_to_word = joblib.load('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/eda/idx_to_word.pkl')

print('The query')
print('========')
for i in keep_one[0][0]:
    #convert i from tensor to index
    print(idx_to_word[i.item()])
print('The positive')
print('========')
for i in keep_one[1][0]:
    #convert i from tensor to index
    print(idx_to_word[i.item()])

The query
rememb
name
author
wrote
cat
hat
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
The positive
mani
children
adult
enjoy
book
dr
seuss
actual
know
anyth
dr
seuss
dr
seuss
born
given
name
theodor
seuss
geisel
grew
springfield
massachusett
left
town
young
man
attend
dartmouth
colleg
new
hampshir
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>


In [174]:
len(embedding_dict)

1244025

In [175]:
# Hyperparameters (example values)
query_sequence_length = SENT_LEN
doc_sequence_length = CONTEXT_LEN
output_dim = 128
embed_dim = 300
lstm_hidden_dim = 64
batch_size = 32

# Instantiate the model
model = TwoTowerModel(query_sequence_length, doc_sequence_length, output_dim, embed_dim, lstm_hidden_dim)


# Define triplet loss
triplet_loss_fn = nn.TripletMarginLoss(margin=1.0, p=2)

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, query, pos_doc, neg_doc):
        # Normalize embeddings to compute cosine similarity
        query_norm = query / query.norm(dim=1, keepdim=True)
        pos_doc_norm = pos_doc / pos_doc.norm(dim=1, keepdim=True)
        neg_doc_norm = neg_doc / neg_doc.norm(dim=1, keepdim=True)
        
        # Compute cosine similarities
        pos_sim = torch.sum(query_norm * pos_doc_norm, dim=1) 
        neg_sim = torch.sum(query_norm * neg_doc_norm, dim=1)
        
        # Compute triplet loss
        loss = torch.max(neg_sim - pos_sim + self.margin, torch.tensor(0)).mean()
        return loss


# Dummy inputs:
# Query and candidate inputs (feature vectors)
#random integers tensor
query = torch.randint(0, 10, (batch_size, query_sequence_length))
doc_positive = torch.randint(0, 20, (batch_size, doc_sequence_length))
doc_negative = torch.randint(0, 20, (batch_size, doc_sequence_length))

# Forward pass
query_embed, positive_embed, candidate_embed = model(query, doc_positive, doc_negative)
query_embed = query_embed.float()
positive_embed = positive_embed.float()
candidate_embed = candidate_embed.float()

# Compute loss (assuming query is the anchor, positive_embed is the positive, and candidate_embed is the negative)
# loss = triplet_loss_fn(query_embed, positive_embed, candidate_embed)
margin = 1.0
loss = TripletLoss(margin)(query_embed, positive_embed, candidate_embed)
print(f"Triplet loss: {loss.item()}")

Triplet loss: 0.9965289235115051


In [183]:
# Hyperparameters (example values)
query_sequence_length = SENT_LEN
doc_sequence_length = CONTEXT_LEN
output_dim = 128
embed_dim = 300
lstm_hidden_dim = 64
batch_size = 32

# Instantiate the model
model = TwoTowerModel(query_sequence_length, doc_sequence_length, output_dim, embed_dim, lstm_hidden_dim)

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, query, pos_doc, neg_doc):
        # Normalize embeddings to compute cosine similarity
        query_norm = query / query.norm(dim=1, keepdim=True)
        pos_doc_norm = pos_doc / pos_doc.norm(dim=1, keepdim=True)
        neg_doc_norm = neg_doc / neg_doc.norm(dim=1, keepdim=True)
        
        # Compute cosine similarities
        pos_sim = torch.sum(query_norm * pos_doc_norm, dim=1) 
        neg_sim = torch.sum(query_norm * neg_doc_norm, dim=1)
        
        # Compute triplet loss
        loss = torch.max(neg_sim - pos_sim + self.margin, torch.tensor(0)).mean()
        return loss

margin = 2.0
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#iterate over batches of the data
losses = []
epochs = 6
for epoch in range(epochs):
    model.train()
    for i, (query, positive, negative) in enumerate(train_loader):
        query_embed, positive_embed, negative_embed = model(query, positive, negative)
        query_embed = query_embed.float()
        positive_embed = positive_embed.float()
        negative_embed = negative_embed.float()
        optimizer.zero_grad()

        loss = TripletLoss(margin)(query_embed, positive_embed, negative_embed)
        loss.backward()

        # Backward pass
        optimizer.step()
        losses.append(loss.item())
        
        if i % 10 == 0:
            print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item()}")
    # Validation
    model.eval()  # Set model to evaluation mode
    valid_loss = 0.0
    with torch.no_grad():  
        for query, positive, negative in val_loader:
            query_embed, positive_embed, negative_embed = model(query, positive, negative)
            query_embed = query_embed.float()
            positive_embed = positive_embed.float()
            negative_embed = negative_embed.float()
            loss = TripletLoss(margin)(query_embed, positive_embed, negative_embed)
            valid_loss += loss.item()
    print(f"Validation loss: {valid_loss / len(val_loader)}")
    # print(f"Validation loss: {loss.item()}")

Epoch 1, Batch 1, Loss: 1.9970744848251343
Validation loss: 2.023116934299469
Epoch 2, Batch 1, Loss: 1.7033318281173706
Validation loss: 2.034234046936035
Epoch 3, Batch 1, Loss: 0.8117779493331909
Validation loss: 2.2215139269828796
Epoch 4, Batch 1, Loss: 0.0247591994702816
Validation loss: 2.2115682721138
Epoch 5, Batch 1, Loss: 0.014092542231082916
Validation loss: 2.2089731216430666
Epoch 6, Batch 1, Loss: 0.005610410124063492
Validation loss: 2.2389997243881226


In [54]:
#embed a query
query = "How to make a cake"
#pad and truncate
query = pad_truncate([query], 4)
query = convert_to_indices(query, word_to_id)

100%|██████████| 1/1 [00:00<00:00, 6955.73it/s]


In [55]:
import numpy as np
query = train[0]['query']
print(query)
query_embed = np.zeros((1, 300))
count_q = 0
for i in tokenize(query, punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True):
    print(i)
    token = word_to_id.get(i, word_to_id['<unk>'])
    query_embed+=embedding_dict.get(token, np.zeros(300))
    print(token)

query_embed = query_embed/max(count_q,1)

print('========')
#embed a positive context
positive_embed = np.zeros((1, 300))
count_p = 0
positive = train[0]['positive']
print(positive)
for i in tokenize(positive, punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True):
    print(i)
    token = word_to_id.get(i, word_to_id['<unk>'])
    positive_embed+=embedding_dict.get(token, np.zeros(300))
    print(token)

positive_embed = positive_embed/max(count_p,1)

print('========')
#embed a negative context
negative_embed = np.zeros((1, 300))
count_p = 0
negative = train[0]['negative']
print(negative)
for i in tokenize(negative, punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True):
    print(i)
    token = word_to_id.get(i, word_to_id['<unk>'])
    negative_embed+=embedding_dict.get(token, np.zeros(300))
    print(token)

negative_embed = negative_embed/max(count_p,1)

what is rba
rba
722428
Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.
sinc
401098
rba
722428
outstand
426933
reput
307411
affect
1188611
secur
734919
npa
740575
scandal
1018751
rba
722428
subsidiari
1036929
involv
70290
bribe
1027549
oversea
1098502
offici
888271
australia
187620
might
396564
win
276869
lucr
541905
note
287077
print
514386
contract
355724
asset
219015
bank
901419
includ
266098
gold
47891
foreign
572029
exchang
309358
reserv
1200897
australia
187620
estim
975472
net
1190646
worth
1157870
billion
1036595
nearli
21564
rba
722428
employ

In [56]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(query_embed, positive_embed))
print(cosine_similarity(query_embed, negative_embed))

[[0.47809076]]
[[0.08634104]]


In [67]:
(query_embed * positive_embed).sum()/np.linalg.norm(query_embed)/np.linalg.norm(positive_embed)

0.47809076495182673

In [30]:
from typing import List
import numpy as np

In [91]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(train, test_size=0.3, random_state=42)

In [92]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
class QueryDataset(Dataset):
    def __init__(self, data):
        self.queries = [d['query'] for d in data]
        self.positives = [d['positive'] for d in data]
        self.negatives = [d['negative'] for d in data]
        print('tokenize')
        self.queries = [tokenize(i,punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True)  for i in tqdm(self.queries)]
        self.positives = [tokenize(i,punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True)  for i in tqdm(self.positives)]
        self.negatives = [tokenize(i,punctuation_map=punctuation_map, stemmer=PorterStemmer(), junk_punctuations=True)  for i in tqdm(self.negatives)]

        print('average pooling')
        self.queries = [self.average_pooling(i) for i in tqdm(self.queries)]
        self.positives = [self.average_pooling(i) for i in tqdm(self.positives)]
        self.negatives = [self.average_pooling(i) for i in tqdm(self.negatives)]
    
    def average_pooling(self, data: List[str]):
        embed = np.zeros((1, 300))
        count = 0
        for d in data:
            token = word_to_id.get(d, word_to_id['<unk>'])
            embed+=embedding_dict.get(token, np.zeros(300))
            count+=1
        # print(data,embed)
        return embed/max(count, 1)
    
    def __len__(self):
        return len(self.queries)
    
    def __getitem__(self, idx):
        return torch.tensor(self.queries[idx], dtype=torch.float32), \
                torch.tensor(self.positives[idx], dtype=torch.float32), \
                torch.tensor(self.negatives[idx], dtype=torch.float32)
    
    def collate_fn(batch):
        queries, positives, negatives = zip(*batch)
        queries = torch.stack(queries)
        positives = torch.stack(positives)
        negatives = torch.stack(negatives)
        return queries, positives, negatives
    

# Wrap in dataset and dataloader
train_dataset = QueryDataset(train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=QueryDataset.collate_fn)

val_dataset = QueryDataset(validation)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=QueryDataset.collate_fn)

tokenize


100%|██████████| 473335/473335 [02:22<00:00, 3327.79it/s]


average pooling
tokenize


100%|██████████| 202858/202858 [01:01<00:00, 3322.41it/s]


average pooling


In [93]:
len(train)

473335

In [94]:
# for i in train_dataset:
#     keep_one = i
#     print(i)
#     print(i[0].shape)
#     print(i[1].shape)
#     break

In [95]:
# import os
# import sys
# src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
# if src_path not in sys.path:
#     sys.path.insert(0, src_path)

In [96]:
class OneTower(nn.Module):
    def __init__(self, embed_dim, med_dim, output_dim):
        super(OneTower, self).__init__()
        self.fc = nn.Linear(embed_dim, med_dim)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(med_dim, output_dim)
    
    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc1(x)
        return x
    
class TwoTowerModel(nn.Module):
    def __init__(self, embed_dim, med_dim, output_dim):
        super(TwoTowerModel, self).__init__()
        self.query_tower = OneTower(embed_dim, med_dim, output_dim)
        self.doc_tower = OneTower(embed_dim, med_dim, output_dim)
        
    def forward(self, query, doc_positive, doc_negative):
        """
        :param query: Tensor of shape (batch_size, query_input_dim)
        :param positive_context: Tensor of shape (batch_size, seq_length) containing token indices
        :param candidate: Tensor of shape (batch_size, query_input_dim) for negative samples
        """
        query_embed = self.query_tower(query)
        positive_embed = self.doc_tower(doc_positive)
        negative_embed = self.doc_tower(doc_negative)
        return query_embed, positive_embed, negative_embed

In [97]:
# Hyperparameters (example values)
import torch.nn.functional as F

output_dim = 64
med_dim = 128
embed_dim = 300
batch_size = 32

# Instantiate the model
model = TwoTowerModel(embed_dim, med_dim,  output_dim)

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, query, pos_doc, neg_doc):
        pos_sim = F.cosine_similarity(query, pos_doc)
        neg_sim = F.cosine_similarity(query, neg_doc)
        
        # Compute triplet loss with margin
        loss = torch.clamp(self.margin - pos_sim + neg_sim, min=0.0)
        return loss.mean()

margin = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#iterate over batches of the data
losses = []
epochs = 30
for epoch in range(epochs):
    model.train()
    for i, (query, positive, negative) in enumerate(val_loader):
        query_embed, positive_embed, negative_embed = model(query, positive, negative)
        query_embed = query_embed.float()
        positive_embed = positive_embed.float()
        negative_embed = negative_embed.float()
        optimizer.zero_grad()

        loss = TripletLoss(margin)(query_embed, positive_embed, negative_embed)
        loss.backward()

        # Backward pass
        optimizer.step()
        losses.append(loss.item())
        
        # if i % 10 == 0:
    print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item()}")
            
    # Validation
    model.eval()  # Set model to evaluation mode
    valid_loss = 0.0
    with torch.no_grad():  
        for query, positive, negative in val_loader:
            query_embed, positive_embed, negative_embed = model(query, positive, negative)
            query_embed = query_embed.float()
            positive_embed = positive_embed.float()
            negative_embed = negative_embed.float()
            loss = TripletLoss(margin)(query_embed, positive_embed, negative_embed)
            valid_loss += loss.item()
    print(f"Validation loss: {valid_loss / len(val_loader)}")
    # print(f"Validation loss: {loss.item()}")

Epoch 1, Batch 6340, Loss: 0.072265625
Validation loss: 0.0713960455665923
Epoch 2, Batch 6340, Loss: 0.05037499591708183
Validation loss: 0.05330694031839961
Epoch 3, Batch 6340, Loss: 0.03793749213218689
Validation loss: 0.03374140936945938
Epoch 4, Batch 6340, Loss: 0.04115623980760574
Validation loss: 0.03158577757120744
Epoch 5, Batch 6340, Loss: 0.08165625482797623
Validation loss: 0.051009222822504745
Epoch 6, Batch 6340, Loss: 0.07856250554323196
Validation loss: 0.04137919181340877
Epoch 7, Batch 6340, Loss: 0.05351562425494194
Validation loss: 0.04645971471578917
Epoch 8, Batch 6340, Loss: 0.05351562425494194
Validation loss: 0.05100312932785189
Epoch 9, Batch 6340, Loss: 0.03804686665534973
Validation loss: 0.0334361846347219
Epoch 10, Batch 6340, Loss: 0.031749993562698364
Validation loss: 0.026691635633005
Epoch 11, Batch 6340, Loss: 0.013093739748001099
Validation loss: 0.014776367540045257
Epoch 12, Batch 6340, Loss: 0.028656240552663803
Validation loss: 0.02092402807410